##### Copyright 2019 The TensorFlow Authors.


In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# [분산 전략을 사용한 모델 저장 및 불러오기](https://www.tensorflow.org/tutorials/distribute/save_and_load)

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/distribute/save_and_load"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />TensorFlow.org에서 보기</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/tutorials/distribute/save_and_load.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩(Colab)에서 실행하기</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/tutorials/distribute/save_and_load.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />깃허브(GitHub)소스 보기</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/tutorials/distribute/save_and_load.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />노트북 다운로드 하기</a>
  </td>

</table>

Note: 이 문서는 텐서플로 커뮤니티에서 번역했습니다. 커뮤니티 번역 활동의 특성상 정확한 번역과 최신 내용을 반영하기 위해 노력함에도 불구하고 [공식 영문 문서](https://www.tensorflow.org/?hl=en)의 내용과 일치하지 않을 수 있습니다. 이 번역에 개선할 부분이 있다면 [tensorflow/docs-l10n](https://github.com/tensorflow/docs-l10n) 깃허브 저장소로 풀 리퀘스트를 보내주시기 바랍니다. 문서 번역이나 리뷰에 참여하려면 [docs-ko@tensorflow.org](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ko)로 메일을 보내주시기 바랍니다.

## 개요

훈련 도중 모델을 저장하고 불러오는 것은 흔히 일어나는 일입니다. 케라스 모델을 저장하고 불러오기 위한 API에는 high-level API와 low-level API, 두 가지가 있습니다. 이 튜토리얼은 `tf.distribute.Strategy`를 사용할 때 어떻게 SavedModel APIs를 사용할 수 있는지 보여줍니다. SavedModel과 직렬화에 관한 일반적인 내용을 학습하려면, [saved model guide](../../guide/saved_model.ipynb)와 [Keras model serialization guide](../../guide/keras/save_and_serialize.ipynb)를 읽어보는 것을 권장합니다. 간단한 예로 시작해보겠습니다:

필요한 패키지 가져오기:

In [2]:
!pip install -q tf-nightly
import tensorflow_datasets as tfds

import tensorflow as tf
tfds.disable_progress_bar()

You should consider upgrading via the '/tmpfs/src/tf_docs_env/bin/python -m pip install --upgrade pip' command.


`tf.distribute.Strategy`를 사용하는 모델과 데이터 준비하기:

In [3]:
mirrored_strategy = tf.distribute.MirroredStrategy()

def get_data():
  datasets, ds_info = tfds.load(name='mnist', with_info=True, as_supervised=True)
  mnist_train, mnist_test = datasets['train'], datasets['test']

  BUFFER_SIZE = 10000

  BATCH_SIZE_PER_REPLICA = 64
  BATCH_SIZE = BATCH_SIZE_PER_REPLICA * mirrored_strategy.num_replicas_in_sync

  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255

    return image, label

  train_dataset = mnist_train.map(scale).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
  eval_dataset = mnist_test.map(scale).batch(BATCH_SIZE)

  return train_dataset, eval_dataset

def get_model():
  with mirrored_strategy.scope():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10)
    ])

    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=['accuracy'])
    return model

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


모델 훈련시키기: 

In [4]:
model = get_model()
train_dataset, eval_dataset = get_data()
model.fit(train_dataset, epochs=2)

Epoch 1/2
938/938 [==============================] - 14s 11ms/step - loss: 0.3931 - accuracy: 0.8903
Epoch 2/2
938/938 [==============================] - 9s 9ms/step - loss: 0.0708 - accuracy: 0.9794


## 모델 저장하고 불러오기
이제 사용할 모델을 가지고 있으므로 API를 이용해 모델을 저장하고 불러오는 방법에 대해 살펴봅시다. 
두 가지 API를 사용 할 수 있습니다:

*   고수준 케라스 `model.save`와 `tf.keras.models.load_model`
*   저수준 케라스 `tf.saved_model.save`와 `tf.saved_model.load`


### 케라스 API

케라스 API들을 이용해 모델을 저장하고 불러오는 예를 소개합니다.

In [5]:
keras_model_path = "/tmp/keras_save"
model.save(keras_model_path)  # save()는 전략 범위를 벗어나 호출되어야 합니다.

/tmpfs/src/tf_docs_env/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:2289: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
/tmpfs/src/tf_docs_env/lib/python3.6/site-packages/tensorflow/python/keras/engine/base_layer.py:1377: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`layer.updates` will be removed in a future version. '


INFO:tensorflow:Assets written to: /tmp/keras_save/assets


INFO:tensorflow:Assets written to: /tmp/keras_save/assets


`tf.distribute.Strategy`없이 모델 복원시키기:

In [6]:
restored_keras_model = tf.keras.models.load_model(keras_model_path)
restored_keras_model.fit(train_dataset, epochs=2)

Epoch 1/2
938/938 [==============================] - 9s 9ms/step - loss: 0.0469 - accuracy: 0.9858
Epoch 2/2
938/938 [==============================] - 8s 9ms/step - loss: 0.0325 - accuracy: 0.9899


모델을 복원시킨 후에는 `compile()`이 이미 저장되기 전에 컴파일 되기 때문에, `compile()`을 다시 호출하지 않고도 모델 훈련을 계속 할 수 있습니다. 그 모델은 텐서플로 표준 `SavedModel`의 프로토 타입에 저장됩니다. 더 많은 정보를 원한다면, [the guide to `saved_model` format](../../guide/saved_model.ipynb)를 참고하세요.

`tf.distribute.strategy`의 범위를 벗어나서 `model.save()` 방법을 호출하는 것은 중요합니다. 범위 안에서 호출하는 것은 지원하지 않습니다. 

이제 모델을 불러와서 `tf.distribute.Strategy`를 사용해 훈련시킵니다:

In [7]:
another_strategy = tf.distribute.OneDeviceStrategy("/cpu:0")
with another_strategy.scope():
  restored_keras_model_ds = tf.keras.models.load_model(keras_model_path)
  restored_keras_model_ds.fit(train_dataset, epochs=2)

Epoch 1/2
938/938 [==============================] - 9s 9ms/step - loss: 0.0481 - accuracy: 0.9850
Epoch 2/2
938/938 [==============================] - 9s 9ms/step - loss: 0.0323 - accuracy: 0.9901


위에서 볼 수 있듯이, 불러오기는 `tf.distribute.Strategy`에서 예상한대로 작동합니다. 여기서 사용된 전략은 이전에 사용된 전략과 같지 않아도 됩니다.

### `tf.saved_model` 형 API

이제 저수준 API에 대해서 살펴봅시다. 모델을 저장하는 것은 케라스 API와 비슷합니다:

In [8]:
model = get_model()  # 새 모델 얻기
saved_model_path = "/tmp/tf_save"
tf.saved_model.save(model, saved_model_path)

INFO:tensorflow:Assets written to: /tmp/tf_save/assets


INFO:tensorflow:Assets written to: /tmp/tf_save/assets


`tf.saved_model.load()`로 불러올 수 있습니다. 그러나 저수준 단계의 API이기 때문에 (따라서 더 넓은 사용범위를 갖습니다), 케라스 모델을 반환하지 않습니다. 대신, 추론하기 위해 사용될 수 있는 기능들을 포함한 객체를 반환합니다. 예를 들어:

In [9]:
DEFAULT_FUNCTION_KEY = "serving_default"
loaded = tf.saved_model.load(saved_model_path)
inference_func = loaded.signatures[DEFAULT_FUNCTION_KEY]

불러와진 객체는 각각 키와 관련된 채, 여러 기능을 포함할 수 있습니다. `"serving_default"`는 저장된 케라스 모델이 있는 추론 기능을 위한 기본 키입니다. 이 기능을 이용하여 추론합니다: 

In [10]:
predict_dataset = eval_dataset.map(lambda image, label: image)
for batch in predict_dataset.take(1):
  print(inference_func(batch))

{'dense_3': <tf.Tensor: shape=(64, 10), dtype=float32, numpy=
array([[-2.31710002e-01,  4.69997153e-02, -5.89058809e-02,
        -7.89453760e-02,  5.00574857e-02,  5.67903817e-02,
        -1.23740345e-01, -1.68794781e-01, -9.27029625e-02,
        -2.51288135e-02],
       [-5.54974042e-02,  4.41712886e-02,  4.67255451e-02,
        -3.14179927e-01, -8.12758040e-03, -1.25043824e-01,
         1.11420020e-01, -1.41085178e-01, -2.66745295e-02,
         1.81214467e-01],
       [-1.52560011e-01,  1.73152894e-01,  1.51429757e-01,
        -1.92452252e-01,  6.56868564e-04, -5.58338277e-02,
        -1.48804402e-02, -1.08949229e-01, -2.73743093e-01,
         2.33208835e-01],
       [-1.47220954e-01,  4.86791804e-02,  4.00493629e-02,
        -1.38356254e-01, -1.55049503e-01,  3.04028448e-02,
        -2.39117034e-02, -1.78863198e-01, -1.02356449e-01,
         1.63152605e-01],
       [-3.83402146e-02,  8.73908177e-02, -2.64412500e-02,
        -1.59144387e-01,  2.08970089e-03, -3.62645015e-02,
        

또한 분산방식으로 불러오고 추론할 수 있습니다:

In [11]:
another_strategy = tf.distribute.MirroredStrategy()
with another_strategy.scope():
  loaded = tf.saved_model.load(saved_model_path)
  inference_func = loaded.signatures[DEFAULT_FUNCTION_KEY]

  dist_predict_dataset = another_strategy.experimental_distribute_dataset(
      predict_dataset)

  # 분산방식으로 기능 호출하기
  for batch in dist_predict_dataset:
    another_strategy.run(inference_func,args=(batch,))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


복원된 기능을 호출하는 것은 단지 저장된 모델로의 정방향 패쓰입니다(예상하기에). 만약 계속해서 불러온 기능을 훈련시키고 싶다면 어떻게 하실건가요? 불러온 기능을 더 큰 모델에 내장시킬 건가요? 일반적인 방법은 이 불러온 객체를 케라스 층에 싸서(wrap) 달성하는 것입니다. 다행히도, [TF Hub](https://www.tensorflow.org/hub)는 이 목적을 위해 [hub.KerasLayer](https://github.com/tensorflow/hub/blob/master/tensorflow_hub/keras_layer.py)을 갖고 있으며, 다음과 같습니다. 

In [12]:
import tensorflow_hub as hub

def build_model(loaded):
  x = tf.keras.layers.Input(shape=(28, 28, 1), name='input_x')
  # KerasLayer로 감싸기
  keras_layer = hub.KerasLayer(loaded, trainable=True)(x)
  model = tf.keras.Model(x, keras_layer)
  return model

another_strategy = tf.distribute.MirroredStrategy()
with another_strategy.scope():
  loaded = tf.saved_model.load(saved_model_path)
  model = build_model(loaded)

  model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
  model.fit(train_dataset, epochs=2)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


Epoch 1/2
938/938 [==============================] - 11s 9ms/step - loss: 0.4051 - accuracy: 0.8846
Epoch 2/2
938/938 [==============================] - 9s 9ms/step - loss: 0.0790 - accuracy: 0.9771


볼 수 있듯이, `hub.KerasLayer`은 `tf.saved_model.load()`로부터 불려온 결과를 또 다른 모델을 만드는데 사용될 수 있는 케라스 층으로 포장(wrap)합니다. 이것은 학습에 매우 유용합니다.

### 어떤 API를 사용해야 할까요?

저장에 관해서, 케라스 모델을 사용하는 경우, 케라스의 `model.save()` API를 사용하는 것을 권장합니다. 저장하려는 모델이 케라스 모델이 아닌 경우, 더 낮은 단계의 API를 선택해야 합니다.

모델을 불러옴에 있어서, 어떤 API를 사용하느냐는 로딩 API에서 얻고자 하는 내용에 따라 결정됩니다. 케라스 모델을 가져올 수 없으면(또는 가져오고 싶지 않다면), `tf.saved_model.load()`를 사용합니다. 그 외의 경우에는, `tf.keras.models.load_model()`을 사용합니다. 케라스 모델을 저장한 경우에만 케라스 모델을 반환 받을 수 있다는 점을 유의하세요. 

API들을 목적에 따라 혼합하고 짜 맞추는 것이 가능합니다. 케라스 모델을 `model.save`와 함께 저장할 수 있고, 저수준 API인, `tf.saved_model.load`로 케라스가 아닌 모델을 불러올 수 있습니다.

In [13]:
model = get_model()

# 케라스의 save() API를 사용하여 모델 저장하기
model.save(keras_model_path) 

another_strategy = tf.distribute.MirroredStrategy()
# 저수준 API를 사용하여 모델 불러오기
with another_strategy.scope():
  loaded = tf.saved_model.load(keras_model_path)

INFO:tensorflow:Assets written to: /tmp/keras_save/assets


INFO:tensorflow:Assets written to: /tmp/keras_save/assets


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


### 주의사항

특별한 경우는 잘 정의되지 않은 입력을 갖는 케라스 모델을 갖고 있는 경우입니다. 예를 들어, 순차 모델은 입력 형태(`Sequential([Dense(3), ...]`) 없이 만들 수 있습니다. 하위 분류된 모델들 또한 초기화 후에 잘 정의된 입력을 갖고 있지 않습니다. 이 경우 모델을 저장하고 불러올 시 저수준 API를 사용해야 하며, 그렇지 않으면 오류가 발생할 수 있습니다.

모델이 잘 정의된 입력을 갖는지 확인하려면, `model.inputs` 이 `None`인지 확인합니다. `None`이 아니라면 잘 정의된 입력입니다. 입력 형태들은 모델이 `.fit`, `.evaluate`, `.predict`에서 쓰이거나 모델을 호출 (`model(inputs)`) 할 때 자동으로 정의됩니다. 

예시를 살펴봅시다:

In [14]:
class SubclassedModel(tf.keras.Model):

  output_name = 'output_layer'

  def __init__(self):
    super(SubclassedModel, self).__init__()
    self._dense_layer = tf.keras.layers.Dense(
        5, dtype=tf.dtypes.float32, name=self.output_name)

  def call(self, inputs):
    return self._dense_layer(inputs)

my_model = SubclassedModel()
# my_model.save(keras_model_path)  # 오류! 
tf.saved_model.save(my_model, saved_model_path)

INFO:tensorflow:Assets written to: /tmp/tf_save/assets


INFO:tensorflow:Assets written to: /tmp/tf_save/assets
